# 신경망(NEURAL NETWORKS)[](https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html#neural-networks)

신경망은 `torch.nn` 패키지를 사용하여 생성할 수 있습니다.

지금까지 `autograd` 를 살펴봤는데요, `nn` 은 모델을 정의하고 미분하는데 `autograd` 를 사용합니다. `nn.Module` 은 계층(layer)과 `output` 을 반환하는 `forward(input)` 메서드를 포함하고 있습니다.

숫자 이미지를 분류하는 신경망을 예제로 살펴보겠습니다:

![convnet](https://tutorials.pytorch.kr/_images/mnist.png)

convnet[](https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html#id5)

이는 간단한 순전파 네트워크(Feed-forward network)입니다. 입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공합니다.

신경망의 일반적인 학습 과정은 다음과 같습니다:

  * 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.

  * 데이터셋(dataset) 입력을 반복합니다.

  * 입력을 신경망에서 전파(process)합니다.

  * 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.

  * 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.

  * 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다: `새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)`


## 신경망 정의하기[](https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html#id1)

이제 신경망을 정의해보겠습니다:

  



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 3x3의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링 (max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # 크기가 제곱수라면 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수만 정의하고 나면, (변화도를 계산하는) `backward` 함수는 `autograd` 를 사용하여 자동으로 정의됩니다. `forward` 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

모델의 학습 가능한 매개변수들은 `net.parameters()` 에 의해 반환됩니다.

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0491, -0.0649,  0.0226,  0.0669,  0.0231,  0.0463,  0.0828, -0.0628,
         -0.0290,  0.1725]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))